# Tugas 2b: Ringkasan Teks Abstraktif

Di <i>notebook</i> ini, Anda akan mengelola tantangan yang timbul dalam ringkasan dokumen besar. Teks input dapat melebihi panjang konteks model yang menghasilkan <i>output</i> halusinasi atau memicu kesalahan kehabisan memori.

Untuk mengurangi masalah ini, <i>notebook</i> ini mendemonstrasikan arsitektur menggunakan pemotongan dan <i>chain</i> perintah dengan kerangka kerja [LangChain] (https://python.langchain.com/docs/get_started/introduction.html), sebuah <i>toolkit</i> yang memungkinkan aplikasi memanfaatkan model bahasa.

Anda akan mempelajari pendekatan yang menangani skenario ketika dokumen pengguna melampaui batas token. Pemotongan membagi dokumen menjadi segmen di bagian ambang panjang konteks sebelum dimasukkan ke model secara berurutan. <i>Chain</i> ini memberikan perintah ke semua potongan dan mempertahankan konteks sebelumnya. Anda menerapkan pendekatan ini untuk merangkum transkrip panggilan, transkrip rapat, buku, artikel, postingan blog, dan konten relevan lainnya.

## Tugas 2b.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tugas 2b.2: Merangkum teks panjang 

### Mengonfigurasi LangChain dengan Boto3

Dalam tugas ini, Anda harus menentukan LLM untuk kelas LangChain Bedrock, dan dapat meneruskan argumen untuk inferensi.

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## Tugas 2b.3: Memuat <i>file</i> teks dengan banyak token

Dalam tugas ini, Anda dapat menemukan <i>file</i> teks [surat CEO Amazon kepada pemegang saham pada tahun 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) di direktori huruf. Sel berikut memuat <i>file </i>teks dan menghitung jumlah token. Anda akan melihat peringatan yang menunjukkan jumlah token dalam <i>file </i>teks melebihi jumlah maksimum token untuk model ini.

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Catatan:** Anda dapat mengabaikan peringatan dan melanjutkan ke sel berikutnya dengan aman.

## Tugas 2b.4: Memisahkan teks panjang menjadi beberapa potongan

Dalam tugas ini, Anda akan memisahkan teks menjadi potongan yang lebih kecil karena terlalu panjang untuk dimuat dalam perintah. `RecursiveCharacterTextSplitter` di LangChain mendukung pemisahan teks panjang menjadi potongan secara rekursif sampai ukuran setiap potongan menjadi lebih kecil dari `chunk_size`. Teks dipisahkan dengan `separators=["\n\n", "\n"]` menjadi beberapa potongan, yang menghindari pemisahan setiap paragraf menjadi beberapa potongan.

Dengan menggunakan 6.000 karakter per potongan, Anda bisa mendapatkan ringkasan untuk setiap bagian secara terpisah. Jumlah token, atau potongan kata, dalam satu potongan bergantung pada teks.

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## Tugas 2b.5: Meringkas potongan dan menggabungkannya

Dalam tugas ini, dengan asumsi bahwa jumlah token sama di dokumen lain, Anda bisa melanjutkan prosesnya. Anda dapat menggunakan `load_summarize_chain` LangChain untuk meringkas teks. `load_summarize_chain` memberikan tiga cara peringkasan: `stuff`, `map_reduce`, dan `refine`.

- `stuff`: menempatkan semua potongan dalam satu perintah. Dengan demikian, batas maksimum token akan tercapai.
- `map_reduce`: merangkum setiap potongan, menggabungkan ringkasan, dan merangkum ringkasan gabungan. Jika terlalu besar, ringkasan gabungan akan menimbulkan kesalahan.
- `refine` merangkum potongan pertama, lalu meringkas potongan kedua dengan ringkasan pertama. Proses yang sama berulang sampai semua potongan diringkas.

Baik map_reduce dan fine memanggil LLM beberapa kali dan membutuhkan waktu untuk mendapatkan ringkasan akhir. Anda dapat mencoba map_reduce di sini.

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Catatan:** Bergantung pada jumlah dokumen Anda, kuota tingkat permintaan Berdrock, dan pengaturan percobaan ulang yang dikonfigurasi - rangkaian di bawah ini mungkin perlu beberapa waktu untuk dijalankan.

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

Kini Anda telah bereksperimen menggunakan pemotongan dan <i>chain</i> perintah dengan kerangka kerja LangChain untuk meringkas dokumen besar sekaligus mengurangi masalah yang timbul dari teks input yang panjang.

### Cobalah sendiri
- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Tugas 3**.